# Compute the Summer Days Climate Index for several years

Starting from the [**Summer Days workflow**](../Demo/Summer_Days_workflow.ipynb) implemented to compute the **Summer Days index** for one input NetCDF data set, we can extend it to compute the index for several years in parallel by exploiting the **iterative and parallel interfaces** made available by the Ophidia Workflow Management System.

**Summer Days index**: number of days where daily maximum temperature $TX > T$ (T is  a reference temperature, e.g. 25°C)


As first step, let's connect to the Ophidia Server

In [ ]:
import sys
from PyOphidia import cube,client
cube.Cube.setclient(read_env=True)

As input files, we can use the daily NetCDF files produced by the CMCC-CM model and related to the tasmax variable for the years 2011-2013. 

In [ ]:
! ls /data

The JSON object associated to the new workflow is quite similar to the previous one.

To exploit the iterative and parallel interfaces we just need to use the **For Statement**, i.e. the pair of operators **OPH_FOR** (with the *parallel* option enabled) and **OPH_ENDFOR**.

The task list includes now the following tasks:

1. **Create container**
 - We use the *oph_createcontainer* (see http://ophidia.cmcc.it/documentation/users/operators/OPH_CREATECONTAINER.html) to create an empty container in order to organize all the datacubes imported and produced during the workflow execution

The **IMPORT**, **MASK**, **COUNT** and **EXPORT** tasks have to be now executed for each of the input years.
So, we use the **OPH_FOR** operator to configure the iterative block and, in particular, to set the number N of loops to be executed. 

2. **Start loop**
 - the **values** argument is an ordered list of N labels (pipe-separated years) to be assigned to cycles 
 - the **parallel** option is enabled
 - each label value can be read in the inner tasks by using @{year} as indicated in the **key** argument

Each inner task depends on OPH_FOR in order to exploit the value of the label associated with the current iteration. 

3. **Import**
 - the *src_path* and the *description* parameters in the *oph_importnc* operator are defined in a parametrized way to get the current value of the *year* key for each iteration
 - the task has a flow dependency from the OPH_FOR task
 
 
4. **Summer Days mask**
 - the *oph_apply* operator (jointly with the *oph_predicate* primitive) is used to identify the summer days: $\{day \mid TX(day) > 298.15\}$ 
 - the task has a flow dependency from the OPH_FOR task and a **single** dependency from the **Import** task
 

5. **Count summer days**
 - count days below the given threshold on yearly basis by using the *oph_reduce2* operator
 - flow dependency from the OPH_FOR task and **single** dependency from **task 4**
 

6. **Export**
 - each Summer Days index is exported to a NetCDF file by using the *oph_exportnc2* operator 
 - flow dependency from the OPH_FOR task and **single** dependency from **task 5**
 
The iterative block ends by means of the **OPH_ENDFOR** operator.

7. **End loop**
 - *oph_endfor* operator
 - flow dependency from **Export** task
 
 
8. **Delete container**
 - *oph_deletecontainer* operator used to empty the workspace
 
 

<img src="Summer_Days_over_several_year.svg" alt="Summer_Days_over_several_year" width="800">

In [ ]:
summer_days = """{
        "name": "Summer Days over several year",
        "author": "CMCC",
        "abstract": "Summer Days index over several years",
        "exec_mode": "sync",
        "ncores": "1",
        "cwd": "/",
        "tasks":
        [
                {
                        "name": "Create container",
                        "operator": "oph_createcontainer",
                        "arguments": 
                        [
                                "container=summer_days",
                                "dim=lat|lon|time",
                                "dim_type=double|double|double",
                                "hierarchy=oph_base|oph_base|oph_time"
                        ]
                },
                {
                        "name": "Start loop",
                        "operator": "oph_for",
                        "arguments": 
                        [
                                "key=year",
                                "values=2011|2012|2013",
                                "parallel=yes"
                        ],
                        "dependencies": [
                                { "task": "Create container"}
                        ]
                },
                {
                        "name": "Import",
                        "operator": "oph_importnc",
                        "arguments":
                        [
                                "src_path=/data/tasmax_day_CMCC-CM_rcp85_r1i1p1_@{year}0101-@{year}1231.nc",                            
                                "measure=$1",
                                "container=summer_days",
                                "import_metadata=yes",
                                "imp_dim=time",
                                "imp_concept_level=d",
                                "vocabulary=CF",
                                "hierarchy=oph_base|oph_base|oph_time",
                                "description=Max Temp @{year}"
                        ],
                        "dependencies": [
                                { "task": "Start loop"}
                        ]
                },   
                {
                        "name": "Summer Days mask",
                        "operator": "oph_apply",
                        "arguments":
                        [
                                "measure_type=auto",
                                "query=oph_predicate(measure,'x-298.15','>0','1','0')",
                                "description=Summer days mask @{year}"
                        ],
                        "dependencies": [
                                { "task": "Import", "type": "single" },
                                { "task": "Start loop"}
                        ]
                },
                {
                        "name": "Count summer days",
                        "operator": "oph_reduce",
                        "arguments":
                        [
                                "operation=sum",
                                "dim=time",
                                "description=Summer Days Count @{year}"
                        ],
                        "dependencies": [
                                { "task": "Summer Days mask", "type": "single" },
                                { "task": "Start loop"}
                        ]
                },
                {
                        "name": "Export",
                        "operator": "oph_exportnc2",
                        "arguments": [
                            "output_name=summer_days_@{year}",
                            "output_path=/data/output"
                        ],
                        "dependencies": [
                            { "task": "Count summer days", "type": "single"},
                            { "task": "Start loop"}
                        ]
                },
                {
                        "name": "End loop",
                        "operator": "oph_endfor",
                        "arguments": [],
                        "dependencies": [
                                { "task": "Export"}
                        ]
                },
                {
                        "name": "Delete container",
                        "operator": "oph_deletecontainer",
                        "arguments": [
                                "container=summer_days",
                                "force=yes"
                        ],
                        "dependencies": [
                                { "task": "End loop"}
                        ]
                }
                
        ]
}"""

Submit the workflow

In [ ]:
cube.Cube.client.wsubmit(summer_days,"tasmax")

Run the *ls* command. As we can see, we have a netCDF file for each year:
- summer_days_2011.nc
- summer_days_2012.nc
- summer_days_2013.nc

In [ ]:
! ls -lh /data/output

Plot a map for each year and create a gif. We just need to install the **netCDF4** and **imageio** libraries

In [ ]:
! pip3 install --user netCDF4 imageio

Run the python script to create the gif map

In [ ]:
! /usr/bin/python3 ./plot_map_cartopy_gif.py

In [ ]:
! ls -lh /data/output | grep "\.gif"

Display the gif inside the notebook

In [ ]:
from IPython.display import Image
with open('/data/output/summer_days_2011_2013.gif','rb') as f:
    display(Image(data=f.read(), format='gif'))